**Please enter the name of the lakehouse where the final Metadata table will be stored in the cell below**

In [1]:
%%configure -f
{
    "defaultLakehouse": {  
        "name": "lakehouse03"
    }
}

StatementMeta(, 29aab01a-b8f0-4d3d-a0b3-12fd77325c14, -1, Finished, Available, Finished)

In [2]:
%pip install semantic-link-labs

StatementMeta(, 29aab01a-b8f0-4d3d-a0b3-12fd77325c14, 7, Finished, Available, Finished)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.5/682.5 kB 10.6 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 24.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 17.0 MB/s eta 0:00:00
  Attempting uninstall: semantic-link-sempy
    Found existing installation: semantic-link-sempy 0.8.0
    Not uninstalling semantic-link-sempy at /home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages, outside environment /nfs4/pyenv-52bce200-8e00-4979-a2e5-f92272651080
    Can't uninstall 'semantic-link-sempy'. No files were found to uninstall.

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



**Please specify variables for lakehouse, warehouse below**

In [3]:
#Specify variables
myLakehouse = 'lakehouse03'
myWorkspace = 'WS_SagarFabric03'

StatementMeta(, 29aab01a-b8f0-4d3d-a0b3-12fd77325c14, 9, Finished, Available, Finished)

In [4]:
myTablePath_semlinklabs = "abfss://"+myWorkspace+"@onelake.dfs.fabric.microsoft.com/"+myLakehouse+".Lakehouse/Tables/ShortcutsMetadata_semanticlinklabs"
myTablePath_semlinklabs_dups = "abfss://"+myWorkspace+"@onelake.dfs.fabric.microsoft.com/"+myLakehouse+".Lakehouse/Tables/ShortcutsMetadata_dups_semanticlinklabs"
#print(myTablePath)

StatementMeta(, 29aab01a-b8f0-4d3d-a0b3-12fd77325c14, 10, Finished, Available, Finished)

In [5]:
import sempy_labs
from sempy_labs import admin, graph, lakehouse
from sempy_labs.tom import connect_semantic_model
import pandas as pd

StatementMeta(, 29aab01a-b8f0-4d3d-a0b3-12fd77325c14, 11, Finished, Available, Finished)

In [6]:
# identify active FSKU capacities and filter the rest
df_cap = admin.list_capacities()
df_cap = df_cap[df_cap["State"]=="Active"]
df_cap = df_cap[(df_cap["State"]=="Active") & (df_cap["Sku"].str.startswith('F'))]
#display(df_cap)

StatementMeta(, 29aab01a-b8f0-4d3d-a0b3-12fd77325c14, 12, Finished, Available, Finished)

In [7]:
# Get only active Fabric workspaces for each capacity
df_ws = pd.DataFrame()
for index, row in df_cap.iterrows():
    #display(row["Capacity Id"])
    df_ws_tmp1 = admin.list_workspaces(capacity=row["Capacity Id"],workspace_state='Active',workspace_type='Workspace')
    #display(df_ws_tmp1)
    df_ws_tmp2 = pd.merge(df_ws_tmp1, df_cap, on='Capacity Id', how='inner')
    #display(df_ws_tmp2)
    df_ws = pd.concat([df_ws, df_ws_tmp2],ignore_index=True)
    #display(df_ws)
df_ws = df_ws[['Id','Name','Capacity Id','Capacity Name', 'Sku', 'Region']].rename(columns={'Id': 'Shortcut Workspace ID', 'Name': 'Shortcut Workspace Name'})
#display(df_ws)

StatementMeta(, 29aab01a-b8f0-4d3d-a0b3-12fd77325c14, 13, Finished, Available, Finished)

In [8]:
# Get lakehouses for the active Fabric workspaces
df_lh = pd.DataFrame()
for index, row in df_ws.iterrows():
    # Select the first row based on index
    first_row = df_ws.iloc[index]
    # Put the first row into another DataFrame
    df_ws_firstrow = pd.DataFrame([first_row])
    #display(df_ws_firstrow)
    df_lh_tmp1 = admin.list_items(capacity=row["Capacity Id"],workspace=row["Shortcut Workspace ID"],type="Lakehouse")
    #display(df_lh_tmp1)
    df_lh_tmp2 = pd.merge(df_lh_tmp1, df_ws_firstrow, left_on='Workspace Id', right_on='Shortcut Workspace ID',how='right')
    #display(df_lh_tmp2)
    df_lh = pd.concat([df_lh, df_lh_tmp2],ignore_index=True)
df_lh = df_lh[['Item Id','Item Name','Last Updated Date','Creator Principal Id','Creator Principal Display Name','Creator Principal Type',
            'Shortcut Workspace ID','Shortcut Workspace Name','Capacity Id_x','Capacity Name', 'Sku', 'Region']]
df_lh = df_lh.rename(columns={'Item Id': 'Lakehouse ID', 'Item Name': 'Lakehouse Name', 'Last Updated Date': 'Lakehouse Last Updated Date', 'Capacity Id_x': 'Capacity ID'})
#display(df_lh)

StatementMeta(, 29aab01a-b8f0-4d3d-a0b3-12fd77325c14, 14, Finished, Available, Finished)

In [9]:
import json

# Get shortcuts for these lakehouses
df_sh = pd.DataFrame()
df_sh_onelake = pd.DataFrame()

for index, row in df_lh.iterrows():
    #display(row["Lakehouse ID"])
    if not pd.isnull(row["Lakehouse ID"]):
        df_sh_tmp1 = lakehouse.list_shortcuts(lakehouse=row["Lakehouse ID"],workspace=row["Shortcut Workspace ID"])

        # Select the first row based on index
        first_row = df_lh.iloc[index]
        # Put the first row into another DataFrame
        df_lh_firstrow = pd.DataFrame([first_row])
        #display(df_lh_firstrow)

        # check if df_sh_tmp1 is empty which means no shortcuts for this lakehouse
        if not df_sh_tmp1.empty:
            df_sh_tmp2 = pd.concat([df_lh_firstrow] * len(df_sh_tmp1), ignore_index=True)
        else:
            df_sh_tmp2 = df_lh_firstrow
        #display(df_sh_tmp2)

        # Combine the two DataFrames
        df_sh_tmp3 = pd.concat([df_sh_tmp1.reset_index(drop=True), df_sh_tmp2.reset_index(drop=True)], axis=1)
        #display(df_sh_tmp3)

    df_sh = pd.concat([df_sh, df_sh_tmp3],ignore_index=True)
    df_sh = df_sh.drop_duplicates()
display(df_sh)

StatementMeta(, 29aab01a-b8f0-4d3d-a0b3-12fd77325c14, 15, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, d8de42da-2729-4b67-9840-7b65a32879ec)

In [10]:
# Rename columns to remove spaces from column names
df_sh = df_sh.rename(columns={'Shortcut Name': 'ShortcutName', 'Shortcut Path': 'ShortcutPath', 'Source Type': 'ShortcutType', 'Source Workspace Id': 'SourceWorkspaceID',
                              'Source Workspace Name': 'SourceWorkspaceName', 'Source Item Id': 'SourceItemID', 'Source Item Name': 'SourceItemName', 'Location': 'SourceLocation',
                              'Source Item Type': 'SourceItemType', 'OneLake Path': 'SourceOneLakePath', 'Connection Id': 'ConnectionID', 'Source Properties Raw': 'SourcePropertiesRaw',
                              'Lakehouse ID': 'ShortcutLakehouseID', 'Lakehouse Name': 'ShortcutLakehouseName', 'Lakehouse Last Updated Date': 'LakehouseLastUpdatedDate', 
                              'Creator Principal Id': 'CreatorPrincipalID', 'Creator Principal Display Name': 'CreatorPrincipalName', 'SubPath': 'SourcePath',
                              'Creator Principal Type': 'CreatorPrincipalType', 'Shortcut Workspace ID': 'ShortcutWorkspaceID', 'Region': 'CapacityRegion',
                              'Shortcut Workspace Name': 'ShortcutWorkspaceName', 'Capacity ID': 'CapacityID', 'Capacity Name': 'CapacityName'
                            })


StatementMeta(, 29aab01a-b8f0-4d3d-a0b3-12fd77325c14, 16, Finished, Available, Finished)

In [11]:
# Convert pandas DataFrame to PySpark DataFrame
pyspark_df_sh = spark.createDataFrame(df_sh)

StatementMeta(, 29aab01a-b8f0-4d3d-a0b3-12fd77325c14, 17, Finished, Available, Finished)

In [12]:
from pyspark.sql.functions import col, get_json_object

#Since Onelake path is not getting correctly assigned, extract it from Source Properties Raw and assign
pyspark_df_sh = pyspark_df_sh.withColumn("SourceOneLakePath", get_json_object(col("SourcePropertiesRaw"), "$.oneLake.path"))
#display(pyspark_df_sh)

StatementMeta(, 29aab01a-b8f0-4d3d-a0b3-12fd77325c14, 18, Finished, Available, Finished)

In [13]:
from pyspark.sql.functions import col
from pyspark.sql import functions as F

# Replace null values with 'NA' in all the columns
pyspark_df_sh = pyspark_df_sh.select([col(c).alias(c) for c in pyspark_df_sh.columns])
pyspark_df_sh = pyspark_df_sh.fillna('NA')
#Add Refresh Date
pyspark_df_sh = pyspark_df_sh.withColumn("LoadDateTime", F.current_timestamp())
#display(pyspark_df_sh)

StatementMeta(, 29aab01a-b8f0-4d3d-a0b3-12fd77325c14, 19, Finished, Available, Finished)

In [14]:
#Internal (OneLake) Duplicate shortcuts
df_dups_int = pyspark_df_sh.filter(pyspark_df_sh.ShortcutType == 'OneLake')
#display(df_dups_int)
df_dups_int = df_dups_int.groupBy("SourceItemID", "SourceONeLakePath","ShortcutLakehouseID").count().filter(col("count") > 1)
#display(df_dups_int)
df_dups_int = pyspark_df_sh.join(df_dups_int, pyspark_df_sh.SourceItemID == df_dups_int.SourceItemID).select(pyspark_df_sh["*"])
# Show duplicate records
#display(df_dups_int)

StatementMeta(, 29aab01a-b8f0-4d3d-a0b3-12fd77325c14, 20, Finished, Available, Finished)

In [15]:
from pyspark.sql import Window

#Add dup column to identify first occurence of duplicate record. This helps in identifying number of unique shortcuts which are duplicated
df_dups_int = df_dups_int.withColumn("dup", F.when(F.row_number().over(Window.partitionBy("SourceItemID", "ShortcutLakehouseID").orderBy(F.lit(1))) == 1, 1).otherwise(0))
#display(df_dups_int)

StatementMeta(, 29aab01a-b8f0-4d3d-a0b3-12fd77325c14, 21, Finished, Available, Finished)

In [16]:
#External (ADLS/S3/GCP) Duplicate shortcuts
df_dups_ext = pyspark_df_sh.filter((pyspark_df_sh.ShortcutType != 'OneLake') & (pyspark_df_sh.ShortcutType != 'NA'))
#display(df_dups_ext)
df_dups_ext = df_dups_ext.groupBy("SourceLocation", "SourcePath", "ShortcutLakehouseID").count().filter(col("count") > 1)
#display(df_dups_ext)
df_dups_ext = pyspark_df_sh.join(df_dups_ext, (pyspark_df_sh.SourceLocation == df_dups_ext.SourceLocation) & 
                                (pyspark_df_sh.SourcePath == df_dups_ext.SourcePath) & 
                                (pyspark_df_sh.ShortcutLakehouseID == df_dups_ext.ShortcutLakehouseID), "inner"
                                ).select(pyspark_df_sh["*"])
# Show duplicate records
#display(df_dups_ext)

StatementMeta(, 29aab01a-b8f0-4d3d-a0b3-12fd77325c14, 22, Finished, Available, Finished)

In [17]:
from pyspark.sql import functions as F
from pyspark.sql import Window

#Add dup column to identify first occurence of duplicate record. This helps in identifying # of duplicated shortcuts
df_dups_ext = df_dups_ext.withColumn("dup", F.when(F.row_number().over(Window.partitionBy("SourceLocation", "SourcePath", "ShortcutLakehouseID")
                         .orderBy(F.lit(1))) == 1, 1).otherwise(0))
#display(df_dups_ext)

StatementMeta(, 29aab01a-b8f0-4d3d-a0b3-12fd77325c14, 23, Finished, Available, Finished)

In [18]:
#Combine internal and external shortcut dups
df_dups = df_dups_int.unionAll(df_dups_ext)
#display(df_dups)

StatementMeta(, 29aab01a-b8f0-4d3d-a0b3-12fd77325c14, 24, Finished, Available, Finished)

In [19]:
if spark.catalog.tableExists("ShortcutsMetadata_semanticlinklabs"):
    drop_stmt = 'DROP TABLE '+myLakehouse+'.ShortcutsMetadata_semanticlinklabs'
    result = spark.sql(drop_stmt)
pyspark_df_sh.write.format("delta").mode("overwrite").save(myTablePath_semlinklabs)

StatementMeta(, 29aab01a-b8f0-4d3d-a0b3-12fd77325c14, 25, Finished, Available, Finished)

In [20]:
if spark.catalog.tableExists("ShortcutsMetadata_dups_semanticlinklabs"):
    drop_stmt = 'DROP TABLE '+myLakehouse+'.ShortcutsMetadata_dups_semanticlinklabs'
    result = spark.sql(drop_stmt)
df_dups.write.format("delta").mode("overwrite").save(myTablePath_semlinklabs_dups)

StatementMeta(, 29aab01a-b8f0-4d3d-a0b3-12fd77325c14, 26, Finished, Available, Finished)

%%sql
SELECT * FROM ShortcutsMetadata_semanticlinklabs